In [63]:
import math
from typing import Annotated

import rich
from fastmcp import Client, FastMCP
from nest_asyncio import apply
from pydantic import Field
from rich import pretty

apply()
pretty.install()

mcp = FastMCP(name="CalculatorServer")


class TrigonometryCalculator:
    @staticmethod
    def add(a: float, b: float):
        """
        Adds two numbers.
        """
        return a + b

    @staticmethod
    def sine(x: float):
        """
        Calculates the sine of an angle in radians.

        :param x: The angle in radians.
        :return: The sine of x.
        """
        return math.sin(x)

    @staticmethod
    def cosine(x: float):
        """
        Calculates the cosine of an angle in radians.

        :param x: The angle in radians.
        :return: The cosine of x.
        """
        return math.cos(x)

    @staticmethod
    def tangent(x: float):
        """
        Calculates the tangent of an angle in radians.

        :param x: The angle in radians.
        :return: The tangent of x.
        """
        return math.tan(x)

    @staticmethod
    def _square(x: float):
        """
        Calculates the square of a number.
        """
        return x * x

    @staticmethod
    def demo(query: str | None = None):
        """
        A demo function.
        """
        return query

    @staticmethod
    def pydantic_field_func(query: Annotated[str, Field(default="test query", description="A demo field")]) -> str:
        """
        A demo function.
        """
        return query


def load_tools_from_module(module_class: object):
    """
    从模块中加载工具
    # 1. 遍历工具类的函数
    # 2. 排除下划线开头的函数
    """
    for name, func in module_class.__dict__.items():
        if name.startswith("_"):
            continue
        if isinstance(func, staticmethod) or isinstance(func, classmethod):
            func = func.__func__
        mcp.tool(func)


load_tools_from_module(TrigonometryCalculator)

In [64]:
client = Client(mcp)

async with client:
    result = await client.list_tools()
    rich.print(result)

[
    Tool(
        name='add',
        title=None,
        description='Adds two numbers.',
        inputSchema={
            'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}},
            'required': ['a', 'b'],
            'type': 'object'
        },
        outputSchema=None,
        annotations=None,
        meta=None
    ),
    Tool(
        name='sine',
        title=None,
        description='Calculates the sine of an angle in radians.\n\n:param x: The angle in radians.\n:return: The 
sine of x.',
        inputSchema={'properties': {'x': {'title': 'X', 'type': 'number'}}, 'required': ['x'], 'type': 'object'},
        outputSchema=None,
        annotations=None,
        meta=None
    ),
    Tool(
        name='cosine',
        title=None,
        description='Calculates the cosine of an angle in radians.\n\n:param x: The angle in radians.\n:return: The
cosine of x.',
        inputSchema={'properties': {'x': {'title': 'X', 'type': 'number'}}, 'required': ['x'], 'type': 'object'},
        outputSchema=None,
        annotations=None,
        meta=None
    ),
    Tool(
        name='tangent',
        title=None,
        description='Calculates the tangent of an angle in radians.\n\n:param x: The angle in radians.\n:return: 
The tangent of x.',
        inputSchema={'properties': {'x': {'title': 'X', 'type': 'number'}}, 'required': ['x'], 'type': 'object'},
        outputSchema=None,
        annotations=None,
        meta=None
    ),
    Tool(
        name='demo',
        title=None,
        description='A demo function.',
        inputSchema={
            'properties': {
                'query': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Query'}
            },
            'type': 'object'
        },
        outputSchema=None,
        annotations=None,
        meta=None
    ),
    Tool(
        name='pydantic_field_func',
        title=None,
        description='A demo function.',
        inputSchema={
            'properties': {
                'query': {
                    'default': 'test query',
                    'description': 'A demo field',
                    'title': 'Query',
                    'type': 'string'
                }
            },
            'type': 'object'
        },
        outputSchema={
            'properties': {'result': {'title': 'Result', 'type': 'string'}},
            'required': ['result'],
            'title': '_WrappedResult',
            'type': 'object',
            'x-fastmcp-wrap-result': True
        },
        annotations=None,
        meta=None
    )
]

In [62]:
import asyncio


async def list_tools(mcp_client: Client):
    async with mcp_client:
        result = await mcp_client.list_tools()
    return result


def reload_tools(client: Client):
    """
    重新加载工具（兼容同步 + 异步环境）
    """
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        # 没有事件循环（同步环境）
        return asyncio.run(list_tools(client))

    # 有事件循环在运行（如 Jupyter/FastAPI 等）
    # 提交到事件循环中执行，并等待返回
    future = asyncio.run_coroutine_threadsafe(list_tools(client), loop)
    return future.result()  # 这个是阻塞等待，适合同步函数中调用


reload_tools(client)

KeyboardInterrupt: 